<h2>Simplest Predictive Algorithm </h>


Simplest predictive algorithm simply does two things:
1) Runs a linear regression on the prices to predict the value of a game in X days, with a floor of $1 <br>
2) During plateau, assesses the frequency of sales (average time between sales), and the median price drop during a sale, to predict the drop. <br>
<br>
To do:<br> 
1) Divide games into "plateau" and "non-plateau" phase, based on the variance of the initial price. <br>
2) Fit simple regression to initial price for "plateau phase", determine price <br>

In [13]:
import os
import json
from pprint import pprint as pp
import pandas as pd
from datetime import datetime
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_pickle('df_steam_db.pkl')

In [60]:
df.head()

,appid,fprice,iprice,price_date,min_fprice,min_iprice,min_iprice_var,min_iprice_slope,min_iprice_intercept,days_since_first_date,decay
0,2430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,297750,"[39.99, 29.99, 29.99, 19.99, 19.99]","[39.99, 39.99, 39.99, 39.99, 39.99]","[1.417120656e+12, 1.434047268e+12, 1.434994431...","[39.99, 29.99, 29.99, 19.99, 19.99]","[39.99, 39.99, 39.99, 39.99, 39.99]",0.000000e+00,0,39.99,"[0.0, 195.909861111, 206.872395833, 266.852592...",False
2,19080,"[9.99, 2.49, 2.49]","[9.99, 9.99, 9.99]","[1.417120656e+12, 1.434049315e+12, 1.434996542...","[9.99, 2.49, 2.49]","[9.99, 9.99, 9.99]",0.000000e+00,0,9.99,"[0.0, 195.933553241, 206.896828704]",False
3,278850,"[4.99, 2.49, 2.49, 2.49, 2.49, 1.49, 1.49]","[4.99, 4.99, 4.99, 4.99, 4.99, 4.99, 4.99]","[1.417120656e+12, 1.417513376e+12, 1.417545207...","[4.99, 2.49, 2.49, 2.49, 2.49, 1.49, 1.49]","[4.99, 4.99, 4.99, 4.99, 4.99, 4.99, 4.99]",7.888609e-31,0,4.99,"[0.0, 4.54537037037, 4.91378472222, 20.9275115...",False
4,367260,"[5.39, 5.39, 4.01, 4.01, 4.01, 4.01, 4.01, 4.01]","[5.99, 5.99, 5.99, 5.99, 5.99, 5.99, 5.99, 5.99]","[1.430513702e+12, 1.431104712e+12, 1.43404599e...","[5.39, 5.39, 4.01, 4.01, 4.01, 4.01, 4.01, 4.01]","[5.99, 5.99, 5.99, 5.99, 5.99, 5.99, 5.99, 5.99]",7.888609e-31,0,5.99,"[0.0, 6.84039351852, 40.882962963, 51.84478009...",False


In [56]:
def timestamptodays(timestamp_vec):
    if not np.isnan(np.sum(timestamp_vec)):
        s_ts = np.array(timestamp_vec) * pow(10,-3)
        s_ts = s_ts - s_ts[0]
        s_ts = s_ts / (60*60*24)
        
    else:
        s_ts = np.nan
    return s_ts

In [57]:
df['days_since_first_date'] = df.price_date.apply(timestamptodays)

In [59]:
df['decay'] = df.min_iprice_var > np.spacing(2)

In [2]:
class PricePred(object):
    '''
    Using a specific appid, pull in the price of the dataframe or database, return predicted price point
    Dataframe, simple regression version    
    '''
    def __init__(self, appid, ideal_price, df):
        self.appid = appid
        self.ideal_price = ideal_price
        
        self.pulldata(df)
        dtw, x, y = self.predict()
        fig = genplot(x,y,dtw)
        
    def pulldata(self,df):
        '''dataframe version'''        
        if appid in df.appid:
            self.row = df.loc[df['appid'] == self.appid]
        else:
            print "appid doesn't exist"
       
    
    def predict(self):
        '''simple regression on initial values'''        
        if self.ideal_price >= self.row.min_price[0]
            x = np.array(self.row.days_since_first_date)
            y = np.array(self.row.min_iprice)
            slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
            
            days_to_wait = (self.ideal_price - intercept) / slope
            days_to_wait = int(days_to_wait)          
            
            return days_to_wait,x,y
            
        else:
            print "price already below value"

    def genplot(self,x,y,days_to_wait):
        '''currently with matplotlib, change later to seaborn'''
        fig, ax = plt.subplots(1,1)
        ax.plot(x,y)
        ax.set_ylabel('Price ($)')
        ax.set_xlabel('Days')
        ax.set_xlim([-1,np.maximum(x) + 1])
        ax.set_ylim([0,np.maximum(y)])
        return fig       

SyntaxError: invalid syntax (<ipython-input-2-8e2c02a67a36>, line 23)